# Restoration and degradation widget
##### The purpose of this notebook is to format the Typology_and_Restoration_Potential table to produce tables that can be used by the API. The data models we need should have the following format:

**Restoration potential:**  
{  
location_id [str]  
indicator [str] - restorable_area, mangrove_area, restoration_potential_score  
value [numeric]  
unit [str] - ha, %  
}  

where:

restorable_area = Tot_Restor  
mangrove_area  = Total_2016  
restoration_potential_score  =  Mean_Score  

**Degradation:**  
{  
location_id [str]  
indicator [str] - degraded_area, lost_area, main_loss_driver  
value [numeric]  
unit [str] - ha, %  
}  

where:

degraded_area = Rest_Area_Dgrd  
lost_area = Area_loss_ha  
main_loss_driver = Loss_Driver


### Steps:
1. Import packages and data
2. Format restoration potential table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table
3. Format degradation table:
- Get countries for the table
- Give each country their corresponding iso code
- Add staging ids
- Create indicators
- Table cleaning
- Save table

#### 1. Import packages and data

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import fiona


In [2]:
path_in = '../../../data/Typology_and_Restoration_Potential/Data'

In [3]:
# Import geodatabase and get Global_Mangrove_Restoration_Jurisdiction_Envelope
gdb_file =f'{path_in}/MOW_Global_Mangrove_Restoration_20190411.gdb'
layers = fiona.listlayers(gdb_file)

In [4]:
layers

['Global_Mangrove_Restoration_Jurisdiction_Envelope',
 'Global_Mangrove_Restoration_Typology']

In [5]:
layer0 = gpd.read_file(gdb_file, driver='FileGDB', layer=0)

In [6]:
layer0.head()

Country  Tot_Restor  Restor_pct  Rest_Area_Loss  Area_loss_pct  \
0       American Samoa     0.00000           0            0.00              0   
1               Angola   580.11556           4          671.43              5   
2             Anguilla     0.08128           4            1.27             59   
3  Antigua and Barbuda    14.15571           2           20.19              2   
4                Aruba    19.38207          36           20.55             38   

   Rest_Area_Dgrd  Area_dgrd_pct  Total_2016  Mean_Score            SOC  \
0            0.00              0       18.74           0       0.000000   
1           68.92              0    13286.05          62  220274.238512   
2            0.00              0        0.87          24     407.670000   
3            0.00              0      886.30          22    9031.556968   
4            0.00              0       33.79          46    9816.581150   

            AGB  People   Fish_Score  Fish_Score_Inv  Shape_Length  \
0      0.000000       0  109000000.0        200000.0  5.350212e+04   
1  30809.263129    3800         -1.0            -1.0  1.383794e+06   
2      6.524752       0    1000000.0       2000000.0  2.097797e+04   
3   1078.042000     500    8000000.0      13000000.0  1.888816e+05   
4   1374.874165       0   71000000.0     119000000.0  3.798229e+04   

     Shape_Area Loss_Driver                                           geometry  
0  1.638363e+07     No Data  MULTIPOLYGON (((-19011686.866 -1613074.241, -1...  
1  2.785231e+10         NPC  MULTIPOLYGON (((1550210.491 -1230695.594, 1549...  
2  1.419573e+06     No Data  MULTIPOLYGON (((-7025249.376 2058315.576, -702...  
3  1.361710e+09     No Data  MULTIPOLYGON (((-6873186.951 1921601.093, -688...  
4  2.166772e+07     No Data  MULTIPOLYGON (((-7781925.061 1393950.107, -778...

In [7]:
# Import locations to get staging ids
locations = pd.read_csv('../../../data/staging_locations.csv')
locations = locations[locations['location_type'] == 'country']
locations = locations[['id', 'name', 'iso']].copy()
locations.head()

id                name  iso
159  1402  Dominican Republic  DOM
160  1401            Colombia  COL
161  1400          Congo, DRC  COD
162  1399           Australia  AUS
163  1398              Angola  AGO

In [8]:
len(locations)

102

In [9]:
# Import gadm file to retrieve ISO codes
gadm = gpd.read_file('../../../data/gadm36_level0_original/gadm36_level0_original.shp')
gadm.head()

GID_0       NAME_0      AREA_KM2  MOL_ID  Shape_Leng  Shape_Area  \
0   ABW        Aruba  1.819384e+02       1    0.963634    0.015131   
1   AFG  Afghanistan  6.438575e+05       2   57.103371   62.749594   
2   AGO       Angola  1.247422e+06       3   73.796528  103.818655   
3   AIA     Anguilla  8.330331e+01       4    1.318321    0.007116   
4   ALA        Åland  1.506261e+03       5   42.232199    0.243769   

                                            geometry  
0  POLYGON ((-69.97820 12.46986, -69.97847 12.469...  
1  POLYGON ((68.52644 31.75435, 68.53852 31.75457...  
2  MULTIPOLYGON (((11.73347 -16.67255, 11.73347 -...  
3  MULTIPOLYGON (((-63.42375 18.58903, -63.42375 ...  
4  MULTIPOLYGON (((21.32195 59.74986, 21.32195 59...

#### 2. Create Restoration Potential table
Get countries for the table


In [10]:
rp = layer0[['Country']]
rp.head()

Country
0       American Samoa
1               Angola
2             Anguilla
3  Antigua and Barbuda
4                Aruba

In [11]:
len(rp['Country'].unique()) # There are more countries here than in locations file so there will be countries with missing IDs

108

Give each country their corresponding iso code


In [12]:
isocodes = gadm[['GID_0', 'NAME_0']]

In [13]:
# Add iso codes to countries in rp dataframe
rp = pd.merge(rp, isocodes, how='left', left_on='Country', right_on='NAME_0')
rp.head()

Country GID_0               NAME_0
0       American Samoa   ASM       American Samoa
1               Angola   AGO               Angola
2             Anguilla   AIA             Anguilla
3  Antigua and Barbuda   ATG  Antigua and Barbuda
4                Aruba   ABW                Aruba

In [14]:
# Check which countries did not match with gadm and have no iso
rp[rp['GID_0'].isnull()]

Country GID_0 NAME_0
12                Bonaire   NaN    NaN
30             East Timor   NaN    NaN
83  Sao Tome and Principe   NaN    NaN

In [15]:
# Add isocodes to those countries
rp.GID_0[12] = 'BES'
rp.GID_0[30] = 'TLS' # East Timor is now called Timor-Leste and the iso is TLS
rp.GID_0[83] = 'STP'

In [16]:
rp = rp.drop(columns='NAME_0').rename(columns={'GID_0':'iso'})
rp.head()

Country  iso
0       American Samoa  ASM
1               Angola  AGO
2             Anguilla  AIA
3  Antigua and Barbuda  ATG
4                Aruba  ABW

Add staging ids

In [17]:
# Add codes from locations
rp = pd.merge(rp, locations, how='left', on='iso').drop(columns={'name'})
rp

Country  iso      id
0          American Samoa  ASM     NaN
1                  Angola  AGO  1398.0
2                Anguilla  AIA     NaN
3     Antigua and Barbuda  ATG  1370.0
4                   Aruba  ABW     NaN
..                    ...  ...     ...
103               Vanuatu  VUT  1394.0
104             Venezuela  VEN  1362.0
105               Vietnam  VNM  1364.0
106  Virgin Islands, U.S.  VIR  1397.0
107                 Yemen  YEM  1366.0

[108 rows x 3 columns]

In [18]:
# Countries with no ID
rp[rp['id'].isnull()]

Country  iso  id
0          American Samoa  ASM NaN
2                Anguilla  AIA NaN
4                   Aruba  ABW NaN
9                Barbados  BRB NaN
25                Curaçao  CUW NaN
28               Dominica  DMA NaN
49              Hong Kong  HKG NaN
83  Sao Tome and Principe  STP NaN

Create indicators

In [19]:
# Add restorable area indicator
ind1 = rp.copy()
ind1['indicator'] = 'restorable_area'
ind1 = pd.merge(ind1,layer0[['Country','Tot_Restor']],on='Country', how='left').rename(columns={'Tot_Restor':'value'})
ind1['unit']='ha'
ind1.head()

Country  iso      id        indicator      value unit
0       American Samoa  ASM     NaN  restorable_area    0.00000   ha
1               Angola  AGO  1398.0  restorable_area  580.11556   ha
2             Anguilla  AIA     NaN  restorable_area    0.08128   ha
3  Antigua and Barbuda  ATG  1370.0  restorable_area   14.15571   ha
4                Aruba  ABW     NaN  restorable_area   19.38207   ha

In [20]:
# Add mangrove area indicator
ind2 = rp.copy()
ind2['indicator'] = 'mangrove_area'
ind2 = pd.merge(ind2,layer0[['Country','Total_2016']],on='Country', how='left').rename(columns={'Total_2016':'value'})
ind2['unit']='ha'
ind2.head()

Country  iso      id      indicator     value unit
0       American Samoa  ASM     NaN  mangrove_area     18.74   ha
1               Angola  AGO  1398.0  mangrove_area  13286.05   ha
2             Anguilla  AIA     NaN  mangrove_area      0.87   ha
3  Antigua and Barbuda  ATG  1370.0  mangrove_area    886.30   ha
4                Aruba  ABW     NaN  mangrove_area     33.79   ha

In [21]:
# Add restoration potential score indicator
ind3 = rp.copy()
ind3['indicator'] = 'restoration_potential_score'
ind3 = pd.merge(ind3,layer0[['Country','Mean_Score']],on='Country', how='left').rename(columns={'Mean_Score':'value'})
ind3['unit']=''
ind3.head()

Country  iso      id                    indicator  value unit
0       American Samoa  ASM     NaN  restoration_potential_score      0     
1               Angola  AGO  1398.0  restoration_potential_score     62     
2             Anguilla  AIA     NaN  restoration_potential_score     24     
3  Antigua and Barbuda  ATG  1370.0  restoration_potential_score     22     
4                Aruba  ABW     NaN  restoration_potential_score     46

In [22]:
# Merge all indicators in same table
restoration = pd.concat([ind1, ind2, ind3], ignore_index=True)
restoration


Country  iso      id                    indicator  \
0          American Samoa  ASM     NaN              restorable_area   
1                  Angola  AGO  1398.0              restorable_area   
2                Anguilla  AIA     NaN              restorable_area   
3     Antigua and Barbuda  ATG  1370.0              restorable_area   
4                   Aruba  ABW     NaN              restorable_area   
..                    ...  ...     ...                          ...   
319               Vanuatu  VUT  1394.0  restoration_potential_score   
320             Venezuela  VEN  1362.0  restoration_potential_score   
321               Vietnam  VNM  1364.0  restoration_potential_score   
322  Virgin Islands, U.S.  VIR  1397.0  restoration_potential_score   
323                 Yemen  YEM  1366.0  restoration_potential_score   

         value unit  
0      0.00000   ha  
1    580.11556   ha  
2      0.08128   ha  
3     14.15571   ha  
4     19.38207   ha  
..         ...  ...  
319   64.00000       
320   57.00000       
321   54.00000       
322   29.00000       
323   27.00000       

[324 rows x 6 columns]

Table cleaning

In [23]:
# Countries with no staging id
restoration[restoration.id.isna()]

Country  iso  id                    indicator      value  \
0           American Samoa  ASM NaN              restorable_area    0.00000   
2                 Anguilla  AIA NaN              restorable_area    0.08128   
4                    Aruba  ABW NaN              restorable_area   19.38207   
9                 Barbados  BRB NaN              restorable_area    0.00000   
25                 Curaçao  CUW NaN              restorable_area   20.95964   
28                Dominica  DMA NaN              restorable_area    0.06000   
49               Hong Kong  HKG NaN              restorable_area   15.09300   
83   Sao Tome and Principe  STP NaN              restorable_area    0.00000   
108         American Samoa  ASM NaN                mangrove_area   18.74000   
110               Anguilla  AIA NaN                mangrove_area    0.87000   
112                  Aruba  ABW NaN                mangrove_area   33.79000   
117               Barbados  BRB NaN                mangrove_area   13.69000   
133                Curaçao  CUW NaN                mangrove_area    9.28000   
136               Dominica  DMA NaN                mangrove_area    1.70000   
157              Hong Kong  HKG NaN                mangrove_area  462.20000   
191  Sao Tome and Principe  STP NaN                mangrove_area    0.67000   
216         American Samoa  ASM NaN  restoration_potential_score    0.00000   
218               Anguilla  AIA NaN  restoration_potential_score   24.00000   
220                  Aruba  ABW NaN  restoration_potential_score   46.00000   
225               Barbados  BRB NaN  restoration_potential_score    0.00000   
241                Curaçao  CUW NaN  restoration_potential_score   33.00000   
244               Dominica  DMA NaN  restoration_potential_score   54.00000   
265              Hong Kong  HKG NaN  restoration_potential_score   36.00000   
299  Sao Tome and Principe  STP NaN  restoration_potential_score    0.00000   

    unit  
0     ha  
2     ha  
4     ha  
9     ha  
25    ha  
28    ha  
49    ha  
83    ha  
108   ha  
110   ha  
112   ha  
117   ha  
133   ha  
136   ha  
157   ha  
191   ha  
216       
218       
220       
225       
241       
244       
265       
299

In [24]:
restoration_final = restoration [['id', 'indicator', 'value', 'unit']].copy()
restoration_final.unit = 'ha'
restoration_final.rename(columns={'id':'location_id'}, inplace=True)


Save table

In [25]:
restoration_final[~restoration_final.location_id.isna()].to_csv('../../../data/RestorationPotential.csv', index = False)

#### Format degradation table


Get countries for the table


In [26]:
# degradation (d) table
d = layer0[['Country']]
d.head()

Country
0       American Samoa
1               Angola
2             Anguilla
3  Antigua and Barbuda
4                Aruba

Give each country their corresponding iso code

In [27]:
# Add iso codes to countries in rp dataframe
d = pd.merge(d, isocodes, how='left', left_on='Country', right_on='NAME_0')
d.head()

Country GID_0               NAME_0
0       American Samoa   ASM       American Samoa
1               Angola   AGO               Angola
2             Anguilla   AIA             Anguilla
3  Antigua and Barbuda   ATG  Antigua and Barbuda
4                Aruba   ABW                Aruba

In [28]:
# Add isocodes to those countries
d.GID_0[12] = 'BES'
d.GID_0[30] = 'TLS' # East Timor is now called Timor-Leste and the iso is TLS
d.GID_0[83] = 'STP'
d = d.drop(columns='NAME_0').rename(columns={'GID_0':'iso'})

# Add codes from locations


Add staging ids


In [29]:
d = pd.merge(d, locations, how='left', on='iso').drop(columns={'name'})

Create indicators

In [30]:
# Create degraded area indicator
ind1 = d.copy()
ind1['indicator'] = 'degraded_area'
ind1 = pd.merge(ind1,layer0[['Country','Rest_Area_Dgrd']],on='Country', how='left').rename(columns={'Rest_Area_Dgrd':'value'})
ind1['unit']='ha'
ind1.head()

Country  iso      id      indicator  value unit
0       American Samoa  ASM     NaN  degraded_area   0.00   ha
1               Angola  AGO  1398.0  degraded_area  68.92   ha
2             Anguilla  AIA     NaN  degraded_area   0.00   ha
3  Antigua and Barbuda  ATG  1370.0  degraded_area   0.00   ha
4                Aruba  ABW     NaN  degraded_area   0.00   ha

In [31]:
# Create lost area indicator
ind2 = d.copy()
ind2['indicator'] = 'lost_area'
ind2 = pd.merge(ind2,layer0[['Country','Rest_Area_Loss']],on='Country', how='left').rename(columns={'Rest_Area_Loss':'value'})
ind2['unit']='ha'
ind2.head()

Country  iso      id  indicator   value unit
0       American Samoa  ASM     NaN  lost_area    0.00   ha
1               Angola  AGO  1398.0  lost_area  671.43   ha
2             Anguilla  AIA     NaN  lost_area    1.27   ha
3  Antigua and Barbuda  ATG  1370.0  lost_area   20.19   ha
4                Aruba  ABW     NaN  lost_area   20.55   ha

In [35]:
# Add mangrove area indicator
ind3 = d.copy()
ind3['indicator'] = 'mangrove_area'
ind3 = pd.merge(ind3,layer0[['Country','Total_2016']],on='Country', how='left').rename(columns={'Total_2016':'value'})
ind3['unit']='ha'
ind3.head()

Country  iso      id      indicator     value unit
0       American Samoa  ASM     NaN  mangrove_area     18.74   ha
1               Angola  AGO  1398.0  mangrove_area  13286.05   ha
2             Anguilla  AIA     NaN  mangrove_area      0.87   ha
3  Antigua and Barbuda  ATG  1370.0  mangrove_area    886.30   ha
4                Aruba  ABW     NaN  mangrove_area     33.79   ha

In [42]:
# Create main loss driver indicator
ind4 = d.copy()
ind4['indicator'] = 'main_loss_driver'
ind4 = pd.merge(ind4,layer0[['Country','Loss_Driver']],on='Country', how='left').rename(columns={'Loss_Driver':'main_loss_driver'})
ind4= ind4.replace({'main_loss_driver':{'No Data':'undefined'}})
ind4 = ind4[['iso', 'main_loss_driver']]
ind4.head()

iso main_loss_driver
0  ASM        undefined
1  AGO              NPC
2  AIA        undefined
3  ATG        undefined
4  ABW        undefined

In [45]:
# Merge all indicators in same table
degradation = pd.concat([ind1, ind2, ind3], ignore_index=True)
degradation_driver = pd.merge(degradation, ind4, on='iso', how = 'left')
degradation_driver

Country  iso      id      indicator      value unit  \
0          American Samoa  ASM     NaN  degraded_area       0.00   ha   
1                  Angola  AGO  1398.0  degraded_area      68.92   ha   
2                Anguilla  AIA     NaN  degraded_area       0.00   ha   
3     Antigua and Barbuda  ATG  1370.0  degraded_area       0.00   ha   
4                   Aruba  ABW     NaN  degraded_area       0.00   ha   
..                    ...  ...     ...            ...        ...  ...   
319               Vanuatu  VUT  1394.0  mangrove_area    1763.60   ha   
320             Venezuela  VEN  1362.0  mangrove_area  280729.61   ha   
321               Vietnam  VNM  1364.0  mangrove_area  159882.87   ha   
322  Virgin Islands, U.S.  VIR  1397.0  mangrove_area     204.97   ha   
323                 Yemen  YEM  1366.0  mangrove_area    1543.70   ha   

    main_loss_driver  
0          undefined  
1                NPC  
2          undefined  
3          undefined  
4          undefined  
..               ...  
319        undefined  
320          Erosion  
321      Commodities  
322        undefined  
323        undefined  

[324 rows x 7 columns]

Table cleaning


In [46]:
degradation_final = degradation_driver[['id', 'indicator', 'value', 'unit', 'main_loss_driver']].copy()
degradation_final.value = np.int64(degradation_final.value)
degradation_final.rename(columns={'id':'location_id'}, inplace=True)
degradation_final.unit= 'ha'


Save table

In [47]:
degradation_final[~degradation_final.location_id.isna()].to_csv('../../../data/Degradation.csv', index = False)